In [ ]:
import sys
sys.path.append('../')

In [ ]:
import os
import argparse
import tensorflow as tf
from tdnn_model import make_tdnn_model, tdnn_config

## from kears model

In [ ]:
config = tdnn_config('M')
eval_model = make_tdnn_model(config, n_labels=1211)
eval_model.load_weights(tf.train.latest_checkpoint("../training/quant_1/M/M"))
eval_model.save("save_model.h5")

In [ ]:
from tdnn_model import StatPooling
converter = tf.lite.TFLiteConverter.from_keras_model_file("save_model.h5",
                                                          input_shapes={'conv2d_input':(1, 500, 1, 65)},
                                                          custom_objects={'StatPooling':StatPooling})
converter.inference_type = tf.int8
tflite_model = converter.convert()
open("tflite_model.tflite", "wb").write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter("tflite_model.tflite")
interpreter.get_tensor_details()

## from frozen graph

In [27]:
ckpt_dir = '../training/quant_1/M/'
model_size = 'M'
batch_size = 32

config = tdnn_config(model_size)
converter = tf.lite.TFLiteConverter.from_frozen_graph(
    os.path.join(ckpt_dir, 'frozen_model.pb'), ['conv2d_input'],
    ['dense/BiasAdd'], {'conv2d_input': (batch_size, 500, 1, 65)}
)

# converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_type = tf.uint8
converter.quantized_input_stats = {'conv2d_input':(128, 127)}
tflite_model = converter.convert()

with open(os.path.join(ckpt_dir, 'quant_aware.tflite'), 'wb') as f:
    f.write(tflite_model)

ConverterError: TOCO failed. See console for info.
2019-06-28 02:42:19.654584: I tensorflow/lite/toco/graph_transformations/graph_transformations.cc:39] Before Removing unused ops: 175 operators, 315 arrays (0 quantized)
2019-06-28 02:42:19.655917: I tensorflow/lite/toco/graph_transformations/graph_transformations.cc:39] After Removing unused ops pass 1: 142 operators, 268 arrays (0 quantized)
2019-06-28 02:42:19.657503: I tensorflow/lite/toco/graph_transformations/graph_transformations.cc:39] Before general graph transformations: 142 operators, 268 arrays (0 quantized)
2019-06-28 02:42:19.658616: I tensorflow/lite/toco/graph_transformations/identify_dilated_conv.cc:202] Replaced sub-network with Dilated Conv2D op outputting "conv2d_1/Conv2D".
2019-06-28 02:42:19.661691: I tensorflow/lite/toco/graph_transformations/identify_dilated_conv.cc:202] Replaced sub-network with Dilated Conv2D op outputting "conv2d_2/Conv2D".
2019-06-28 02:42:19.667714: I tensorflow/lite/toco/graph_transformati

In [ ]:
interpreter = tf.lite.Interpreter(os.path.join(ckpt_dir, 'quant_aware.tflite'))
interpreter.get_tensor_details()

In [ ]:
%%bash 

toco \
  --graph_def_file=../training/quant_1/M/frozen_model.pb \
  --output_file=tflite_model.tflite \
  --input_format=TENSORFLOW_GRAPHDEF --output_format=TFLITE \
  --inference_type=QUANTIZED_UINT8 \
  --input_shape="1,500, 1,65" \
  --input_array=conv2d_input \
  --output_array=dense/BiasAdd \
  --std_dev_value=128 --mean_value=128 